In [ ]:
import os,sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
from scipy.spatial import ConvexHull
from scipy.io import loadmat
sys.path.append('./src/')
import PyGradLimitPDE as pglpde

In [ ]:
'''
geometry setting
'''
def addNodeInLines(node):
    '''
    在node 构成的几何中以最短边为单位插入节点
    '''
    edge_vec = np.roll(node, -1, axis=0) - node
    edge_len = np.sqrt(np.sum(edge_vec**2, axis=1))
    minLen = np.min(edge_len) if np.min(edge_len)<=0.01 else 0.01 
    
    addNodeDcit={}
    for ith,elen in enumerate(edge_len):
        segNum = int(elen/minLen)
        addNodeDcit[ith] = [ node[ith]+ (node[(ith+1)%len(node)]-node[ith])/segNum*i for i in range(1,segNum)]

    nodeExtend = node
    addlength =0
    for k,v in addNodeDcit.items():
        if v:
            nodeExtend =  np.insert(nodeExtend,k+1+addlength,np.array(v),axis=0)
            addlength+=len(v)
    index = np.isin(nodeExtend,node).all(1) # 扩展后，原有节点node在nodeExtend中的位置标记
    return nodeExtend ,index

def remove_large_angle(node):
    angles_vec_ccw = np.roll(node, -1, axis=0) - node    # counterclockwise and clockwise  angle vector
    angles_vec_cc = np.roll(node, 1, axis=0) - node
    norm_ccw = np.linalg.norm(angles_vec_ccw, axis=1)
    norm_cc  = np.linalg.norm(angles_vec_cc, axis=1)     # Calculate the magnitudes of vectors and normlize vectors
    cos_theta = np.sum(angles_vec_ccw * angles_vec_cc, 
                        axis=1) / (norm_cc * norm_ccw)   # calculate cosine theta , dot product = norm_A * norm_B *cos_theta
    theta_rad = np.degrees(np.arccos(cos_theta))         # Calculate the angle between two vector
    node = node[theta_rad<170,:]
    return node

def node_angle_len(node):
    """
    每组点组成一个闭环，生成每个点处的角度值及其相临边的边长均值
    Args:
        node (array): 节点坐标
    Returns:
        edge_len(array):每条边的长度
        theta_rad(array , range in [0,360]): 每个角度的值，其中凹边对应的角大于180度,
                    数值在(0,360)间
    """
    edge_vec_1 = np.roll(node, -1, axis=0) - node # calculate edge vector and length, mean value of counterclockwise and clockwise
    edge_len_1 = np.sqrt(np.sum(edge_vec_1**2, axis=1))
    edge_vec_2 = np.roll(node, 1, axis=0) - node
    edge_len_2 = np.sqrt(np.sum(edge_vec_2**2, axis=1))
    edge_len = (edge_len_1 + edge_len_2) / 2
    
    angles_vec_ccw = np.roll(node, -1, axis=0) - node # counterclockwise and clockwise  angle vector
    angles_vec_cc = np.roll(node, 1, axis=0) - node
    norm_ccw = np.linalg.norm(angles_vec_ccw, axis=1) # Calculate the magnitudes of vectors and normlize vectors
    norm_cc  = np.linalg.norm(angles_vec_cc, axis=1)
    
    cos_theta = np.sum(angles_vec_ccw * angles_vec_cc,
                       axis=1) / (norm_cc * norm_ccw) # calculate cosine theta , dot product = norm_A * norm_B *cos_theta
    theta_rad = np.degrees(np.arccos(cos_theta))      # Calculate the angle between two vector
    # 判断夹角是否为凹角，如果是则转换为负值 .负数对应凹边的角，用360加去后得到大于180度的凹边角
    cross_product = np.cross(angles_vec_cc, angles_vec_ccw)
    theta_rad[cross_product > 0] = -theta_rad[cross_product > 0]
    theta_rad[theta_rad < 0] = 360 + theta_rad[theta_rad < 0]
    large_rad_index = np.argwhere(theta_rad > 170).squeeze() #r如果有大于170的则删去
    return theta_rad, edge_len, large_rad_index

def scale_to_target(arr, target_range=(0.095, 0.096)):
    """将权重数据缩放到指定空间
    Args:
        arr (array): 权重数据
        target_range (float, optional): 缩放到的空间指定值. Defaults to (0.05, 0.15).
    Returns:
        array: 缩放后的权重
    """
    min_val = np.min(arr)
    max_val = np.max(arr)
    scaled_arr = (arr - min_val) / (max_val - min_val)  # 将数据缩放到[0, 1]范围
    # scaled_arr = np.sqrt(scaled_arr) # 使用平方根函数进行非线性缩放
    range_min, range_max = np.array(target_range)# /len(arr) # 缩放区间范围
    # scaled_arr = np.log(1+scaled_arr) * (range_max -range_min) + range_min  # 将数据缩放到目标范围
    scaled_arr = scaled_arr * (range_max -range_min) + range_min  # 将数据缩放到目标范围
    return scaled_arr

def geometry_spacing(geom_nodes):
    # node angle range in [0,pi];
    spacing_range=0.001
    node_angle=1/softmax(1/node_angle_len(geom_nodes)[0]) * softmax(node_angle_len(geom_nodes)[1]) #长边与小小角度获得较大的权重

    # 将权重缩放到指定空间
    spacing=scale_to_target(node_angle,target_range=(spacing_range,spacing_range)) 
    return spacing

def geometry_weights(geom_nodes):
    # node angle range in [0,pi];
    weights_range=1
    node_angle=softmax(1/node_angle_len(geom_nodes)[0]) # * softmax(node_angle_len(v)[1]) #长边与小小角度获得较大的权重

    # 将权重缩放到指定空间
    weights= scale_to_target(node_angle,target_range=(weights_range,weights_range*1.5))
                    
    return weights

def isNarrow(ps):
    '''
    #* 判断由ps构成的凸几何是否有很短的边
    '''
    edgeVec =  np.roll(ps, -1, axis=0) - ps 
    edgeLen =  np.sqrt(np.sum(edgeVec**2, axis=1))
    minEdgeLen = edgeLen.min()
    return True if minEdgeLen < edgeLen.mean()/5 else False

# np.random.seed(89)
numPoints=np.random.randint(10,35)
xr,yr = np.random.uniform(0.3,0.7),np.random.uniform(0.15,0.40)
xy=np.c_[xr*np.random.rand(numPoints)+(1-xr)/2,yr*np.random.rand(numPoints)+(1-yr)/2]
chull=ConvexHull(xy)

# if not isNarrow(chull.points[chull.vertices]):
inner=chull.points[chull.vertices] 
inner = remove_large_angle(inner)
# else:
#     raise Exception('narrow geom')


In [ ]:
# 绘制inner 几何图像
plt.figure(figsize=(6,6))
plt.plot(inner[:,0],inner[:,1],'k-')
plt.axis('equal')
plt.axis([0,1,0,1])

In [ ]:
inner_nodes , inner_index = addNodeInLines(inner)

outer_edge_nodes_n = 2
outer_bottom = np.c_[np.linspace(0,1,outer_edge_nodes_n-1, endpoint=False),np.zeros(outer_edge_nodes_n-1)]
outer_right = np.c_[np.ones(outer_edge_nodes_n-1),np.linspace(0,1,outer_edge_nodes_n-1, endpoint=False)]
outer_top = np.c_[np.linspace(1,0,outer_edge_nodes_n-1, endpoint=False),np.ones(outer_edge_nodes_n-1)]
outer_left = np.c_[np.zeros(outer_edge_nodes_n-1),np.linspace(1,0,outer_edge_nodes_n-1, endpoint=False)]
outer_nodes = np.r_[outer_bottom,outer_right,outer_top,outer_left]
_,index = np.unique(outer_nodes,axis=0,return_index=True)
outer_nodes = outer_nodes[sorted(index),:]
outer_nodes , outer_index = addNodeInLines(outer_nodes)

inner_spacing = np.zeros(len(inner_index))
inner_spacing[inner_index] = geometry_spacing(inner_nodes[inner_index])
inner_spacing[~inner_index] = inner_spacing[inner_index].mean()

inner_nodal_psi  = np.zeros(len(inner_index))
inner_nodal_psi[inner_index] = geometry_weights(inner_nodes[inner_index])
inner_nodal_psi[~inner_index]=inner_nodal_psi[inner_index].mean()/100

inner_linear_psi = np.zeros(len(inner_index))
inner_linear_psi[inner_index] = geometry_weights(inner_nodes[inner_index])
inner_linear_psi[~inner_index] = inner_linear_psi[inner_index].mean()/20


outer_spacing = np.ones(len(outer_index))
outer_spacing[outer_index] = 1
outer_spacing[~outer_index] = 0.5

outer_nodal_psi= np.ones(len(outer_index))
outer_nodal_psi[outer_index] = 1
outer_nodal_psi[~outer_index] = 0

outer_linear_psi= np.ones(len(outer_index))
outer_linear_psi[outer_index] = 1
outer_linear_psi[~outer_index] = 0.1

In [ ]:
# plot outer nodes
%matplotlib widget
plt.plot(outer_nodes[:,0],outer_nodes[:,1],'r-')
plt.plot(inner_nodes[:,0],inner_nodes[:,1],'b-')
plt.axis('equal')
plt.axis([0,1,0,1])


In [ ]:
print(f'inner:{inner.shape}; inner_nodes:{inner_nodes.shape}')
print(f'min inner edge length: {np.sqrt(np.sum((np.roll(inner, -1, axis=0) - inner)**2,axis=1)).min()}')
print(f'max inner edge length: {np.sqrt(np.sum((np.roll(inner, -1, axis=0) - inner)**2,axis=1)).max()}')
print(f'inner spacing: {inner_spacing} ')
print(f'inner nodal psi: {inner_nodal_psi} ')
print(f'inner linear psi:{inner_linear_psi}')
print(f'output spacing: {outer_spacing}')
print(f'output nodal psi: {outer_nodal_psi}')
print(f'output linear psi: {outer_linear_psi}')

In [ ]:

inner_boundary = pglpde.Boundary(len(inner_nodes),inner_nodes,inner_spacing,inner_nodal_psi,inner_linear_psi)
outer_boundary = pglpde.Boundary(len(outer_nodes),outer_nodes,outer_spacing,outer_nodal_psi,outer_linear_psi)
##* 均匀网格设置
background_grid = pglpde.UniformSquareGrid([0., 0.], 1., 512, outer_spacing.mean())
##* 组装背景网格与边界
gradient_limit_pde = pglpde.GradientLimitPDE(background_grid, [inner_boundary, outer_boundary])
##* 求解
result = pglpde.solve(gradient_limit_pde)
result = np.array(result).T

In [ ]:
from matplotlib import pyplot as plt
%matplotlib widget

dx= 512
X,Y= np.meshgrid(np.linspace(0,1,dx),np.linspace(0,1,dx))
XY= np.stack([X,Y],axis=2).reshape(-1,2)
fig,[ax1,ax2] = plt.subplots(1,2,figsize=(10,5))

ax1.pcolormesh(X,Y,result,cmap='jet')

ilen =len(inner_nodes)

for i in range(ilen):
    ax2.plot(inner_nodes[[i%ilen,(i+1)%ilen],0],inner_nodes[[i%ilen,(i+1)%ilen],1],'y-+')
ax2.plot(inner_nodes[inner_index,0],inner_nodes[inner_index,1],'r*')
ax2.axis([0,1,0,1])